In [11]:
import urllib.request
import h5py

In [17]:
import h5py
import urllib.request
import io

zenodo_url = "https://zenodo.org/records/10119316/files/val.h5"

# Read the file into memory
with urllib.request.urlopen(zenodo_url) as response:
    file_data = response.read()  # Read all data into memory

# Wrap the data in a BytesIO object to enable seeking
file_buffer = io.BytesIO(file_data)

# Open the HDF5 file
with h5py.File(file_buffer, "r") as h5f:
    print(list(h5f.keys()))  # List available datasets inside the HDF5 file


KeyboardInterrupt: 